In [28]:
import pandas as pd

### 1. 날씨 데이터 전처리

In [29]:
weather = pd.read_csv("./weather.csv")
weather.head()

,지점,지점명,일시,평균기온,일강수량,평균 풍속,평균 상대습도,일 최심적설
0,108,서울,2018-10-01,15.4,0.0,2.9,55.1,0.0
1,108,서울,2018-10-02,15.9,0.0,1.6,61.4,0.0
2,108,서울,2018-10-03,17.3,0.0,1.3,58.0,0.0
3,108,서울,2018-10-04,19.3,0.0,1.3,56.5,0.0
4,108,서울,2018-10-05,16.9,36.5,1.7,84.1,0.0


In [30]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지점       365 non-null    int64  
 1   지점명      365 non-null    object 
 2   일시       365 non-null    object 
 3   평균기온     365 non-null    float64
 4   일강수량     365 non-null    float64
 5   평균 풍속    365 non-null    float64
 6   평균 상대습도  365 non-null    float64
 7   일 최심적설   365 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 22.9+ KB


In [31]:
# 쓸데없는 컬럼 제거
weather.drop(columns=["지점", "지점명"], inplace=True)

In [32]:
# 컬럼명 정리
weather.rename(columns={"일시" : "일자",
                        "평균기온" : "기온",
                       "일강수량" : "강수량",
                       "평균 풍속" : "풍속",
                       "평균 상대습도" : "습도",
                       "일 최심적설" : "적설량"}, inplace=True)

In [33]:
# 배달데이터와 일자 형식 맞추기
weather["일자"] = weather["일자"].str.replace("-", "")

In [44]:
weather

,일자,기온,강수량,풍속,습도,적설량
0,20181001,15.4,0.0,2.9,55.1,0.0
1,20181002,15.9,0.0,1.6,61.4,0.0
2,20181003,17.3,0.0,1.3,58.0,0.0
3,20181004,19.3,0.0,1.3,56.5,0.0
4,20181005,16.9,36.5,1.7,84.1,0.0
...,...,...,...,...,...,...
360,20190926,22.9,0.0,1.9,59.3,0.0
361,20190927,22.9,0.0,1.9,57.5,0.0
362,20190928,23.7,0.0,1.7,58.9,0.0
363,20190929,22.6,0.0,1.7,60.6,0.0


### 2. 배달 데이터 전처리

In [50]:
# 강남구 데이터
delivery = pd.read_csv("./delivery.csv")
delivery.head()

,일자,요일,시간대,시도,시군구,읍면동,통화건수
0,20181001,월,0,서울특별시,강남구,논현동,5
1,20181001,월,0,서울특별시,강남구,세곡동,5
2,20181001,월,0,서울특별시,강남구,삼성동,7
3,20181001,월,0,서울특별시,강남구,역삼동,5
4,20181001,월,0,서울특별시,강서구,등촌동,5


In [51]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383050 entries, 0 to 383049
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   일자      383050 non-null  int64 
 1   요일      383050 non-null  object
 2   시간대     383050 non-null  int64 
 3   시도      383050 non-null  object
 4   시군구     383050 non-null  object
 5   읍면동     383050 non-null  object
 6   통화건수    383050 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 20.5+ MB


In [52]:
# 날짜별로 groupby
delivery = delivery.groupby(["일자", "요일"]).sum().drop(columns="시간대").reset_index()

In [53]:
# '일자' 컬럼 데이터형 일치시키기
delivery["일자"] = delivery["일자"].astype('str')
delivery

,일자,요일,통화건수
0,20181001,월,9681
1,20181002,화,13692
2,20181003,수,15403
3,20181004,목,10091
4,20181005,금,17680
...,...,...,...
360,20190926,목,8908
361,20190927,금,12561
362,20190928,토,13828
363,20190929,일,12188


In [54]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      365 non-null    object
 1   요일      365 non-null    object
 2   통화건수    365 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 8.7+ KB


### 3. 날씨+배달 데이터 합치기

In [56]:
df = pd.merge(weather, delivery, on="일자")
df = df[["일자", "요일", "기온", "강수량", "풍속", "습도", "적설량", "요일", "통화건수"]]

In [57]:
df

,일자,요일,기온,강수량,풍속,습도,적설량,요일,통화건수
0,20181001,월,15.4,0.0,2.9,55.1,0.0,월,9681
1,20181002,화,15.9,0.0,1.6,61.4,0.0,화,13692
2,20181003,수,17.3,0.0,1.3,58.0,0.0,수,15403
3,20181004,목,19.3,0.0,1.3,56.5,0.0,목,10091
4,20181005,금,16.9,36.5,1.7,84.1,0.0,금,17680
...,...,...,...,...,...,...,...,...,...
360,20190926,목,22.9,0.0,1.9,59.3,0.0,목,8908
361,20190927,금,22.9,0.0,1.9,57.5,0.0,금,12561
362,20190928,토,23.7,0.0,1.7,58.9,0.0,토,13828
363,20190929,일,22.6,0.0,1.7,60.6,0.0,일,12188


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 364
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      365 non-null    object 
 1   요일      365 non-null    object 
 2   기온      365 non-null    float64
 3   강수량     365 non-null    float64
 4   풍속      365 non-null    float64
 5   습도      365 non-null    float64
 6   적설량     365 non-null    float64
 7   요일      365 non-null    object 
 8   통화건수    365 non-null    int64  
dtypes: float64(5), int64(1), object(3)
memory usage: 28.5+ KB
